In [14]:
from caveclient import CAVEclient
from nglui import parser
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

client = CAVEclient('minnie65_public')

In [12]:
url = 'https://neuroglancer.neuvue.io/?json_url=https://global.daf-apis.com/nglstate/api/v1/5298478177583104'

# This splits the url string at every "/" and then takes the last one (the number part) and turns it into an integer.
state_id = int(url.split('/')[-1])

# Download the state
state = client.state.get_state_json(state_id)

Get the annotation dataframes

In [29]:
anno_df = parser.annotation_dataframe(state)
anno_df = anno_df.query('anno_type != "bbox"')

anno_df.head()

,layer,anno_type,point,pointB,linked_segmentation,tags,group_id,description
0,synapses,point,"[159850, 244135, 24485]",NaN,"[864691135494808976, 864691135412658258]",[],None,None
1,synapses,point,"[207682, 218096, 23052]",NaN,"[864691135494808976, 864691135809943756]",[1],None,None
2,synapses,point,"[158738, 245466, 24608]",NaN,"[864691135494808976, 864691135412734546]",[],None,None
3,synapses,point,"[185052, 221602, 23773]",NaN,"[864691135494808976, 864691135608549060]",[1],None,None
4,synapses,point,"[162868, 242334, 24122]",NaN,"[864691135494808976, 864691136349347298]",[],None,None


,layer,anno_type,point,pointB,linked_segmentation,tags,group_id,description,mushroom,stubby,thin,idk
0,synapses,point,"[159850, 244135, 24485]",NaN,"[864691135494808976, 864691135412658258]",[],None,None,False,False,False,False
1,synapses,point,"[207682, 218096, 23052]",NaN,"[864691135494808976, 864691135809943756]",[1],None,None,True,False,False,False
2,synapses,point,"[158738, 245466, 24608]",NaN,"[864691135494808976, 864691135412734546]",[],None,None,False,False,False,False
3,synapses,point,"[185052, 221602, 23773]",NaN,"[864691135494808976, 864691135608549060]",[1],None,None,True,False,False,False
4,synapses,point,"[162868, 242334, 24122]",NaN,"[864691135494808976, 864691136349347298]",[],None,None,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
185,synapses,line,"[163607.890625, 241916.5, 24050.640625]","[163832.3125, 241981.21875, 24075.3046875]","[864691135494808976, 864691135494808976]",[1],None,None,True,False,False,False
186,synapses,line,"[179853.9375, 211669.875, 20255.5]","[179936.390625, 211543.859375, 20236.314453125]","[864691135494808976, 0]",[4],None,None,False,False,False,True
187,synapses,line,"[182791.640625, 214859.15625, 22003.978515625]","[182815.703125, 214761.765625, 21985.974609375]","[864691135494808976, 864691135494808976]",[1],None,None,True,False,False,False
188,synapses,line,"[167748.953125, 208178.125, 21278.275390625]","[167959.203125, 208099.03125, 21300.740234375]","[864691135494808976, 864691135209420632, 86469...",[1],None,None,True,False,False,False


In [30]:
# Compute the mapping from tag numbers to strings
tag_dictionary = parser.tag_dictionary(state, layer_name='synapses')
tag_dictionary

{1: 'mushroom', 2: 'stubby', 3: 'thin', 4: 'idk'}

In [31]:
# this adds one column per tag category with a true/false value in each row

for k, v in tag_dictionary.items():
    new_col = [np.isin(k, tag_list) for tag_list in anno_df.tags]
    anno_df[v] = new_col

Get line lengths

In [58]:
anno_df

,layer,anno_type,point,pointB,linked_segmentation,tags,group_id,description,mushroom,stubby,thin,idk
0,synapses,point,"[159850, 244135, 24485]",NaN,"[864691135494808976, 864691135412658258]",[],None,None,False,False,False,False
1,synapses,point,"[207682, 218096, 23052]",NaN,"[864691135494808976, 864691135809943756]",[1],None,None,True,False,False,False
2,synapses,point,"[158738, 245466, 24608]",NaN,"[864691135494808976, 864691135412734546]",[],None,None,False,False,False,False
3,synapses,point,"[185052, 221602, 23773]",NaN,"[864691135494808976, 864691135608549060]",[1],None,None,True,False,False,False
4,synapses,point,"[162868, 242334, 24122]",NaN,"[864691135494808976, 864691136349347298]",[],None,None,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
225,synapses,bbox,"[163607.890625, 241916.5, 24050.640625]","[163832.3125, 241981.21875, 24075.3046875]","[864691135494808976, 864691135494808976]",[1],None,None,True,False,False,False
226,synapses,bbox,"[179853.9375, 211669.875, 20255.5]","[179936.390625, 211543.859375, 20236.314453125]","[864691135494808976, 0]",[4],None,None,False,False,False,True
227,synapses,bbox,"[182791.640625, 214859.15625, 22003.978515625]","[182815.703125, 214761.765625, 21985.974609375]","[864691135494808976, 864691135494808976]",[1],None,None,True,False,False,False
228,synapses,bbox,"[167748.953125, 208178.125, 21278.275390625]","[167959.203125, 208099.03125, 21300.740234375]","[864691135494808976, 864691135209420632, 86469...",[1],None,None,True,False,False,False


In [39]:
ptA = np.vstack(anno_df.query('anno_type=="line"')['point'])
ptB = np.vstack(anno_df.query('anno_type=="line"')['pointB'])

In [45]:
np.linalg.norm((ptA-ptB) * [4,4,40], axis=1) / 1000

array([1.02849406, 0.67993464, 0.96167749, 1.00427093, 1.82536215,
       1.20500621, 1.53116324, 1.77769127, 0.69473467, 3.67969601,
       2.05468901, 1.71604205, 0.98614713, 1.79544419, 2.05230174,
       1.12698087, 1.75768488, 1.48173387, 1.76155363, 0.9954707 ,
       2.46727078, 2.25564334, 1.57551924, 1.54693575, 0.89150326,
       1.35379542, 1.31294037, 0.90330916, 1.17368124, 1.08285326,
       2.21698345, 0.88223484, 1.70395888, 1.06983793, 1.27619425,
       1.35873643, 0.97559808, 0.82440768, 1.27076547, 1.6555428 ])

Linking synapses to skeleton locations:

In [46]:
# Reuse code to download neurons

def load_neuron(root_id, client):
    from skeleton_plot.skel_io import load_mw
    cloud_dir = 'https://storage.googleapis.com/allen-minnie-phase3/minniephase3-emily-pcg-skeletons/minnie_all/v661/meshworks'
    df = client.materialize.views.single_neurons(pt_root_id=root_id).query()
    if len(df)==0:
        raise ValueError(f'No pre-computed skeleton for root id {root_id}')
    else:
        return load_mw(cloud_dir, f'{root_id}_{df.iloc[0]["id"]}.h5')

In [48]:
nrn = load_neuron(864691135494808976, client)

The `client.materialize.views` interface is experimental and might experience breaking changes before the feature is stabilized.


In [57]:
syn_df = nrn.anno.post_syn.df
syn_df['dist_to_root'] = nrn.distance_to_root(nrn.anno.post_syn.mesh_index) / 1000

In [62]:
def split_column_into_components(column_name, df):
    """
    Split column name into three components with columns: column_name_x, column_name_y, column_name_z
    """
    df = df.copy() # Copy so you don't change the dataframe in place
    df[f'{column_name}_x'] = df[column_name].apply(lambda x: x[0]) # extract the first coordinate of the position
    df[f'{column_name}_y'] = df[column_name].apply(lambda x: x[1]) # extract the second coordinate of the position
    df[f'{column_name}_z'] = df[column_name].apply(lambda x: x[2]) # extract the third coordinate of the position
    return df

In [64]:
syn_df = split_column_into_components('ctr_pt_position', syn_df)

In [83]:
# split anno_df into points and lines as separate dataframes
point_df = anno_df.query('anno_type == "point"')
line_df = anno_df.query('anno_type == "line"')

In [70]:
point_df = split_column_into_components('point', point_df)

In [80]:
syn_category_df = syn_df.merge(
    point_df[[
        'mushroom',
        'stubby',
        'thin',
        'idk',
        'point_x',
        'point_y',
        'point_z',
    ]],
    left_on=[
        'ctr_pt_position_x', 'ctr_pt_position_y', 'ctr_pt_position_z'
    ],
    right_on=[
        'point_x', 'point_y', 'point_z'
    ],
).drop(
    columns=[
        'ctr_pt_position_x', 'ctr_pt_position_y', 'ctr_pt_position_z', 'point_x', 'point_y', 'point_z',
    ]
) # This last bit just cleans up these columns because we don't need them after merging, since we still have the original positions

Adding spine length to our dataframe!

In [ ]:
from scipy.spatial import kdtree

# Assign an "is spine" value to each point
point_df['is_spine'] = point_df[['mushroom', 'stubby', 'thin', 'idk']].sum(axis=1)

# Build a k-d tree to look up the closest spine point to the line start
synapse_points = np.vstack(point_df['point']) * [4,4,40]
synapse_kdt = kdtree.KDTree(synapse_points)

# Get the points of each line start
line_starts = np.vstack(line_df['pointB']) * [4,4,40]

#inds from this is the index of the synapse closest to the start of the line point
ds, inds = synapse_kdt.query(line_starts)

# compute the spine length as the distance from point to point B
line_df['spine_length'] = np.linalg.norm(np.vstack(line_df['point']) - np.vstack(line_df['pointB']) * [4,4,40], axis=1)

# This is a convoluted by effective way to assign the spine lenght value to the rows in "point_df" that are associated with the closest synapse location
point_df.loc[
    point_df.iloc[inds].index,
    'spine_length',
] = line_df['spine_length'].values